In [56]:
import os

old_folder = r'D:\Pu\20220208-P_brain_M1_nonclear\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

In [57]:
os.path.isfile(before_position_file), os.path.isfile(after_position_file)

(True, True)

# Calculate rotation matrix

In [58]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [59]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99989048, -0.01479949],
        [ 0.01479949,  0.99989048]]),
 array([   50.27017055, -1393.00834786]))

# Translate 60x positions

In [60]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [61]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-2513.90835986  -372.5480477 ]
 [-2516.86825835  -572.52614401]
 [-2519.82815685  -772.50424031]
 [-2522.78805535  -972.48233661]
 [-2322.80995904  -975.44223511]
 [-2319.85006055  -775.46413881]
 [-2316.89016205  -575.4860425 ]
 [-2313.93026355  -375.5079462 ]
 [-2310.97036506  -175.5298499 ]
 [-2110.99226875  -178.4897484 ]
 [-2113.95216725  -378.4678447 ]
 [-2116.91206575  -578.445941  ]
 [-2119.87196424  -778.4240373 ]
 [-2122.83186274  -978.40213361]
 [-1922.85376644  -981.3620321 ]
 [-1919.89386794  -781.3839358 ]
 [-1916.93396945  -581.4058395 ]
 [-1913.97407095  -381.4277432 ]
 [-1911.01417245  -181.44964689]
 [-1711.03607615  -184.40954539]
 [-1713.99597465  -384.38764169]
 [-1716.95587314  -584.36573799]
 [-1719.91577164  -784.3438343 ]
 [-1722.87567014  -984.3219306 ]
 [-1522.89757383  -987.2818291 ]
 [-1519.93767534  -787.30373279]
 [-1516.97777684  -587.32563649]
 [-1514.01787834  -387.34754019]
 [-1511.05797985  -187.36944389]
 [-1311.07988354  -190.32934238]
 [-1314.03

In [62]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\Alignment\translated_positions_all.txt


# Manual adjust positions

In [63]:
manual_positions_before_file = os.path.join(os.path.dirname(old_folder), '60x_positions_before.txt')
print(manual_positions_before_file)

D:\Pu\20220208-P_brain_M1_nonclear\60x_positions_before.txt


In [64]:
manual_positions = np.loadtxt(manual_positions_before_file, delimiter=',')
print(manual_positions)
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 2))

[[-3834.    -602.92]
 [-3101.46  4229.73]
 [ 6538.3   2639.23]
 [ 5862.38 -2320.71]]
translated manual positions:
[[-3792.23 -1939.12]
 [-2988.25  2882.16]
 [ 6626.91  1149.17]
 [ 5877.66 -3800.22]]


In [68]:
manual_real_positions = [
    [-3816, -1941.5],
    [-3016, 2880],
    [6598, 1151.5],
    [5856.5, -3797],
]
manual_shifts = np.array(manual_real_positions) - translated_manual_positions[:len(manual_real_positions)]
manual_shifts

array([[-23.76715442,  -2.37893721],
       [-27.7476951 ,  -2.15845247],
       [-28.91337038,   2.33091404],
       [-21.16280137,   3.22443596]])

In [69]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([-25.39775532,   0.25449008])

In [70]:
adjusted_new_positions = new_positions + manual_shift

In [71]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20220215-P_brain_CTP11-1000_CTP12_from0208\Alignment\adjusted_translated_positions_all.txt
